In [1]:
import re
import pandas as pd

In [9]:
###
# Get the courses from a file, into a dict.
###

courses = {}
fileRead = '' ## DEBUG

with open('./pages/HIS-2019-fall.txt', 'r') as course_file:
    # Read course file into one big string
    data = course_file.read()
    
    fileRead = data ## DEBUG
    
    # Break down to courses
    data = data.split('Collapse section ')
    
    # Remove erroneous space items
    del data[0]
    
    # Key the courses into the `courses` object
    for course in data:
        courses[course[0:8]] = course

In [10]:
def handle_section(section):
    sectionCode = section[0].split('-')[0]
    
    sectionData = {
        'admin': {},
        'days': []
    }
    
    ## Handling admin stuff
    sectionData['admin']['code'] = sectionCode
    sectionData['admin']['type'] = section[0].split('-')[1].split('\n')[0]
    sectionData['admin']['duration'] = section[0].split('-')[1].split('\n')[1]
    sectionData['admin']['isOpen'] = 'Open' in section[5]
    
    ## Handling days
    
    ### Helper function to extract datetime details
    def handle_section_datetime(datetime):
        return {
            'weekday': datetime[:2],
            'times': {
                'start': datetime[3:].split(' - ')[0],
                'end': datetime[3:].split(' - ')[1]
            }
        }

    ### Extract the relevant details in a very hacky way :)
    def mini_day_extractor(sectionToHandle, index):
        return {
            **handle_section_datetime(sectionToHandle[1].split('\n')[index]),
            'location': sectionToHandle[2].split('\n')[index],
            'prof': sectionToHandle[3].split('\n')[index],
            'dates': {
                'start': sectionToHandle[4].split('\n')[index].split(' - ')[0],
                'end': sectionToHandle[4].split('\n')[index].split(' - ')[1]
            }
        }
    
    ### Given the number of days (counted by the number of `\n` in the datetime string...), extract them
    for index in range(0, section[1].count('\n') + 1):
        sectionData['days'].append(mini_day_extractor(section, index))
            
    return {sectionCode: sectionData}

In [11]:
###
# Get the sections from each course, adding as a list to the dict item.
###

for courseCode, courseStr in courses.items():
    sections = []
    
    if ' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n' in courseStr:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    else:
        sectionContainer = courseStr.split(' \n\t\t\n\t\n\t\t\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')[1]
    
    sectionData = sectionContainer.split('\n\t\n\t\t\n\t\n  \tSection \tDays & Times \tRoom \tInstructor \tMeeting Dates \tStatus\nDetails\n\t\n')
    
    for section in sectionData:
        sectionInfo = handle_section(section.split('\n\t\n'))
        
        sections.append(sectionInfo)
        
    courses[courseCode] = sections

In [12]:
## Clean badly processed courses :)
coursesToDelete = {}

for courseCode, courseObj in courses.items():
    if isinstance(courseObj, str):
        coursesToDelete[courseCode] = courseObj
        
for course in coursesToDelete:
    courses.pop(course, None)
        
print(str(len(coursesToDelete)) + " courses removed due to badly formed data.")

0 courses removed due to badly formed data.


In [13]:
courses

{'HIS 1101': [{'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '161 Louis Pasteur (CBY) C03',
     'prof': 'Damien-Claude Bélanger',
     'times': {'end': '14:20', 'start': '13:00'},
     'weekday': 'We'},
    {'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '550 Cumberland (TBT) 333',
     'prof': 'Damien-Claude Bélanger',
     'times': {'end': '12:50', 'start': '11:30'},
     'weekday': 'Fr'}]}],
 'HIS 1110': [{'admin': {'code': 'A00',
    'duration': 'FullSess.',
    'isOpen': True,
    'type': 'LEC'},
   'days': [{'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '125 University (MNT) 202',
     'prof': 'Eric Allina',
     'times': {'end': '12:50', 'start': '11:30'},
     'weekday': 'Mo'},
    {'dates': {'end': '2019-12-03', 'start': '2019-09-04'},
     'location': '125 University (MNT) 202',
     'prof': 'M

In [16]:
pd.read_json(courses)

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [21]:
pd.io.json.json_normalize(courses)

,HIS 1101,HIS 1110,HIS 1111,HIS 1500,HIS 1501,HIS 1520,HIS 2100,HIS 2101,HIS 2103,HIS 2105,...,HIS 4535,HIS 4551,HIS 4762,HIS 4770,HIS 5103,HIS 5503,HIS 6332,HIS 7338,HIS 7738,HIS 8900
0,"[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'B00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...",...,"[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'LEC', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur...","[{'admin': {'code': 'A00', 'type': 'SEM', 'dur..."
